In [19]:
import requests #library to handle request
from bs4 import BeautifulSoup
import pandas as pd  # library to process data as dataframes
import numpy as np # library to handle data in a vectorized manner 
from geopy.geocoders import Nominatim     # convert an address into latitude and longitude values  
import folium      # map rendering library
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans   # import k-means from clustering stage

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [20]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_T')
soup = BeautifulSoup(page.content, 'html.parser')

In [21]:
table = soup.find('table', attrs={'class':'wikitable sortable'})


In [69]:
df = pd.read_html(str(table))[0]
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,T1A,Medicine Hat,Central Medicine Hat,50.036460,-110.679250
1,T2A,Calgary,"Penbrooke Meadows, Marlborough",51.049680,-113.964320
2,T3A,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158
3,T4A,Airdrie,East Airdrie,51.272450,-113.986980
4,T5A,Edmonton,"West Clareview, East Londonderry",53.5899,-113.4413


In [84]:
df = df[df.Borough != 'Not assigned']  # Dropping the rows where Borough is 'Not assigned'
df = df[df.Latitude != 'Not assigned']
df = df[df.Longitude != 'Not assigned']
df = df[df.Neighborhood != 'Not assigned']
df = df.groupby(['Postal Code','Borough'], sort=False).agg(', '.join)  # Combining the neighborhoods with same Postal Code
df.reset_index(inplace=True)   


# Replacing the neighborhoods which are 'Not assigned' with names of Borough
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned',df['Borough'], df['Neighborhood'])


In [117]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,T1A,Medicine Hat,Central Medicine Hat,50.036460,-110.679250
1,T2A,Calgary,"Penbrooke Meadows, Marlborough",51.049680,-113.964320
2,T3A,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158
3,T4A,Airdrie,East Airdrie,51.272450,-113.986980
4,T5A,Edmonton,"West Clareview, East Londonderry",53.5899,-113.4413


In [118]:
df.shape

(97, 5)

In [123]:
df['Latitude'] = pd.to_numeric(df['Latitude'])

In [124]:
df['Longitude'] = pd.to_numeric(df['Longitude'])

In [125]:
# Get the latitude and longitude coordinates of Toronto
address = "Calgary, Alberta"

geolocator = Nominatim(user_agent="calgary_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Calgary city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Calgary city are 51.0534234, -114.0625892.


In [126]:
# create map of Toronto using latitude and longitude coordinates
map_calgary = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(
        df['Latitude'], 
        df['Longitude'], 
        df['Borough'], 
        df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_calgary)  

map_calgary


In [91]:
df_calgary = df[df['Borough'].str.contains("Calgary")].reset_index(drop=True)
df_calgary.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,T2A,Calgary,"Penbrooke Meadows, Marlborough",51.049680,-113.964320
1,T3A,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158
2,T2B,Calgary,"Forest Lawn, Dover, Erin Woods",51.0318,-113.9786
3,T3B,Calgary,"Montgomery, Bowness, Silver Springs, Greenwood",51.0809,-114.1616
4,T2C,Calgary,"Lynnwood Ridge, Ogden, Foothills Industrial, G...",50.9878,-114.0001


In [92]:
CLIENT_ID = 'K33SGF45U0LCTPWW2P2DTBSMZPWVSZU4YIS5KHC4FQISDKH4' # your Foursquare ID
CLIENT_SECRET = 'SVEKAXAOHPB1EOM3AP32F1TLL3AEDZLN0TJRW2BET51ELN4L' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K33SGF45U0LCTPWW2P2DTBSMZPWVSZU4YIS5KHC4FQISDKH4
CLIENT_SECRET:SVEKAXAOHPB1EOM3AP32F1TLL3AEDZLN0TJRW2BET51ELN4L


In [93]:
neighborhood_name = df_calgary.loc[0, 'Neighborhood']
print('The first neighborhood is ', neighborhood_name)

The first neighborhood is  Penbrooke Meadows, Marlborough


In [94]:
neighborhood_latitude = df_calgary.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_calgary.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Penbrooke Meadows, Marlborough are 51.049680, -113.964320.


In [95]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 2000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# get the result to a json file
results = requests.get(url).json()

In [96]:
results

{'meta': {'code': 200, 'requestId': '5f1898a751d07f34bca903e6'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Forest Lawn - Forest Heights',
  'headerFullLocation': 'Forest Lawn - Forest Heights, Calgary',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 69,
  'suggestedBounds': {'ne': {'lat': 51.06768001800002,
    'lng': -113.93574047968525},
   'sw': {'lat': 51.031679981999986, 'lng': -113.99289952031475}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c604af13986e21e4bd8934f',
       'name': 'Bob Bahan Aquatic & Fitness Centre',
       'location': {'address': '4812 14 Ave SE',
        'lat': 51.04160911564571,
        'lng': -113.96431909261312,
        'l

In [97]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [98]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

C:\Users\schor\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,Bob Bahan Aquatic & Fitness Centre,Gym / Fitness Center,51.041609,-113.964319
1,Pho Dau Bo,Vietnamese Restaurant,51.037592,-113.963598
2,Olive Garden,Italian Restaurant,51.052937,-113.982449
3,Shawarma Palace,Falafel Restaurant,51.037969,-113.979086
4,Atlas Pizza and Sports Bar,Pizza Place,51.052481,-113.941859
...,...,...,...,...
64,Liquor Barn -transcanada,Liquor Store,51.064254,-113.956193
65,Border Crossing,Bar,51.037197,-113.981370
66,Tomoka Coffee,Coffee Shop,51.037764,-113.982446
67,Bubblicious,Juice Bar,51.061645,-113.983903


In [99]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [101]:
calgary_venues = getNearbyVenues(names=df_calgary['Neighborhood'],
                                   latitudes=df_calgary['Latitude'],
                                   longitudes=df_calgary['Longitude']
                                  )

In [102]:
calgary_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158,Petro-Canada,51.128068,-114.138057,Gas Station
1,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158,Edgemont City,51.126473,-114.138997,Asian Restaurant
2,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158,Friends Cappuccino Bar & Bake Shop,51.126370,-114.138676,Café
3,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158,Mac's,51.128309,-114.137902,Convenience Store
4,"Forest Lawn, Dover, Erin Woods",51.0318,-113.9786,Bonasera Pizza And Sports Bar,51.029893,-113.982543,Bar


In [103]:
calgary_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Braeside, Cedarbrae, Woodbine",8,8,8,8,8,8
"Brentwood, Collingwood, Nose Hill",2,2,2,2,2,2
"Bridgeland, Greenview, Zoo, YYC",22,22,22,22,22,22
"City Centre, Calgary Tower",27,27,27,27,27,27
"Connaught, West Victoria Park",41,41,41,41,41,41
"Cranston, Auburn Bay, Mahogany",4,4,4,4,4,4
"Dalhousie, Edgemont, Hamptons, Hidden Valley",4,4,4,4,4,4
"Discovery Ridge, Signal Hill, West Springs, Christie Estates, Patterson, Cougar Ridge",5,5,5,5,5,5
"Douglas Glen, McKenzie Lake, Copperfield, East Shepard",12,12,12,12,12,12


In [104]:
# one hot encoding
calgary_onehot = pd.get_dummies(calgary_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
calgary_onehot['Neighborhood'] = calgary_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [calgary_onehot.columns[-1]] + list(calgary_onehot.columns[:-1])
calgary_onehot = calgary_onehot[fixed_columns]

calgary_onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,...,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Vape Store,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Shop,Yoga Studio
0,"Dalhousie, Edgemont, Hamptons, Hidden Valley",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Dalhousie, Edgemont, Hamptons, Hidden Valley",0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Dalhousie, Edgemont, Hamptons, Hidden Valley",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Dalhousie, Edgemont, Hamptons, Hidden Valley",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Forest Lawn, Dover, Erin Woods",0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [105]:
calgary_grouped = calgary_onehot.groupby('Neighborhood').mean().reset_index()
calgary_grouped.head()

,Neighborhood,American Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,...,Tapas Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Vape Store,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Shop,Yoga Studio
0,"Braeside, Cedarbrae, Woodbine",0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000
1,"Brentwood, Collingwood, Nose Hill",0.0,0.0,0.500000,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00000
2,"Bridgeland, Greenview, Zoo, YYC",0.0,0.0,0.045455,0.0,0.0,0.0,0.000000,0.090909,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,0.0,0.00000
3,"City Centre, Calgary Tower",0.0,0.0,0.000000,0.0,0.0,0.0,0.074074,0.000000,0.037037,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037037,0.0,0.00000
4,"Connaught, West Victoria Park",0.0,0.0,0.000000,0.0,0.0,0.0,0.024390,0.000000,0.048780,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.02439


In [122]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = calgary_grouped['Neighborhood']

for ind in np.arange(calgary_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(calgary_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Braeside, Cedarbrae, Woodbine",Pizza Place,Convenience Store,Gym,Coffee Shop,Hockey Rink,Pub,Pharmacy,Ice Cream Shop,Yoga Studio,Eastern European Restaurant
1,"Brentwood, Collingwood, Nose Hill",Asian Restaurant,Dog Run,Yoga Studio,Flea Market,Department Store,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dry Cleaner
2,"Bridgeland, Greenview, Zoo, YYC",Bank,Fast Food Restaurant,Seafood Restaurant,Middle Eastern Restaurant,Dim Sum Restaurant,Coffee Shop,Chinese Restaurant,Noodle House,Indian Restaurant,Pharmacy
3,"City Centre, Calgary Tower",Restaurant,Pub,Mediterranean Restaurant,Bakery,Sushi Restaurant,Coffee Shop,Pizza Place,Eastern European Restaurant,Nightclub,Indie Movie Theater
4,"Connaught, West Victoria Park",Coffee Shop,French Restaurant,Bar,Restaurant,Brewery,Middle Eastern Restaurant,Mediterranean Restaurant,Pub,Ice Cream Shop,Hotel
5,"Cranston, Auburn Bay, Mahogany",Cosmetics Shop,Pizza Place,Electronics Store,Liquor Store,Yoga Studio,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run
6,"Dalhousie, Edgemont, Hamptons, Hidden Valley",Convenience Store,Asian Restaurant,Gas Station,Café,Flea Market,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
7,"Discovery Ridge, Signal Hill, West Springs, Ch...",Convenience Store,Bar,Pizza Place,Gas Station,Vietnamese Restaurant,Department Store,Dim Sum Restaurant,Diner,Discount Store,Falafel Restaurant
8,"Douglas Glen, McKenzie Lake, Copperfield, East...",Pub,Dry Cleaner,Business Service,Bus Stop,Intersection,Food Truck,Forest,Gas Station,Bar,Thrift / Vintage Store
9,"Elbow Park, Britannia, Parkhill, Mission",Japanese Restaurant,Coffee Shop,Deli / Bodega,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant


In [107]:
# set number of clusters
kclusters = 5

calgary_grouped_clustering = calgary_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(calgary_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [138]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Labels', kmeans.labels_)

calgary_merged = df_calgary

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
calgary_merged = calgary_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

 # check the last columns!
calgary_merged['Lab


In [139]:
calgary_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Labels,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,T2A,Calgary,"Penbrooke Meadows, Marlborough",51.049680,-113.964320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,T3A,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158,0.0,0.0,Convenience Store,Asian Restaurant,Gas Station,Café,Flea Market,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
2,T2B,Calgary,"Forest Lawn, Dover, Erin Woods",51.0318,-113.9786,0.0,0.0,Convenience Store,Bar,Martial Arts Dojo,Smoke Shop,Gift Shop,Falafel Restaurant,Department Store,Dim Sum Restaurant,Diner,Discount Store
3,T3B,Calgary,"Montgomery, Bowness, Silver Springs, Greenwood",51.0809,-114.1616,0.0,0.0,Coffee Shop,Pizza Place,Food Court,Scenic Lookout,Yoga Studio,Falafel Restaurant,Department Store,Dim Sum Restaurant,Diner,Discount Store
4,T2C,Calgary,"Lynnwood Ridge, Ogden, Foothills Industrial, G...",50.9878,-114.0001,0.0,0.0,Convenience Store,Diner,Clothing Store,Pizza Place,Fast Food Restaurant,Department Store,Dim Sum Restaurant,Discount Store,Dog Run,Donut Shop
5,T3C,Calgary,"Rosscarrock, Westgate, Wildwood, Shaganappi, S...",51.0388,-114.098,0.0,0.0,Sandwich Place,Indian Restaurant,Pub,Mexican Restaurant,Sports Bar,Candy Store,Pizza Place,Fast Food Restaurant,Breakfast Spot,Bookstore
6,T2E,Calgary,"Bridgeland, Greenview, Zoo, YYC",51.0632,-114.0614,0.0,0.0,Bank,Fast Food Restaurant,Seafood Restaurant,Middle Eastern Restaurant,Dim Sum Restaurant,Coffee Shop,Chinese Restaurant,Noodle House,Indian Restaurant,Pharmacy
7,T3E,Calgary,"Lakeview, Glendale, Killarney, Glamorgan",51.0227,-114.1342,0.0,0.0,Vietnamese Restaurant,Wine Shop,Vape Store,Coffee Shop,Yoga Studio,Donut Shop,Factory,Electronics Store,Eastern European Restaurant,Dry Cleaner
8,T2G,Calgary,"Inglewood, Burnsland, Chinatown, East Victoria...",51.0415,-114.0599,0.0,0.0,Coffee Shop,Pub,Performing Arts Venue,Italian Restaurant,Theater,Hotel,Deli / Bodega,Restaurant,Asian Restaurant,New American Restaurant
9,T3G,Calgary,"Hawkwood, Arbour Lake, Citadel, Ranchlands, Ro...",51.1147,-114.1796,3.0,3.0,Pizza Place,Pub,Yoga Studio,Falafel Restaurant,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run


In [140]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        calgary_merged['Latitude'], 
        calgary_merged['Longitude'], 
        calgary_merged['Neighborhood'], 
        calgary_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


TypeError: list indices must be integers or slices, not float